In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import os
from dotenv import load_dotenv
import pymongo
import sqlite3
import mysql.connector
from mysql.connector import errorcode

In [2]:
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',  # or your server IP
    'database': 'stocks',
    'raise_on_warnings': True
}

In [3]:
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

In [4]:
def create_table(cursor):

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS StockData (
            id INT AUTO_INCREMENT PRIMARY KEY,
            Open FLOAT,
            High FLOAT,
            Low FLOAT,
            Close FLOAT,
            Volume BIGINT,
            Dividends FLOAT,
            Stock_Splits FLOAT,
            Date DATETIME,
            ticker VARCHAR(10)
        )
        ''')

    conn.commit()

create_table(cursor)


In [ ]:
def insert_stockdata_db(tickers):

    stock_data = yf.Ticker(tickers)

    stock_history = stock_data.history(period="3mo")

    stock_history["Date"] = stock_history.index

    stock_history["ticker"] = tickers

    stock_history.reset_index(drop=True,inplace=True)

    sample_data = list(stock_history.itertuples(index=False, name=None))

    insert_query = '''
    INSERT INTO StockData (
        Open, High, Low, Close, Volume, Dividends, Stock_Splits, Date, ticker
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''

    cursor.executemany(insert_query, sample_data)

    conn.commit()

In [6]:
tickers = ["AAPL", "NVDA", "TSLA", "MSFT", "GOOGL"]

In [7]:
for ticker in tickers:

    insert_stockdata_db(ticker)